### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install -U sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, https://pip.repos.neuron.amazonaws.com
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c7/ec/74e608c9e3458c16464265c38018f4e26b4232d5a46674a277af15bd56ac/sagemaker-2.173.0.tar.gz (854 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 105.6 MB/s eta 0:00:0000:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.173.0-py2.py3-none-any.whl size=1163282 sha256=01f26cec98522a11e6a0ae218eb317db81da31fe20bd306441afbac6e07cd5d1
  Stored in directory: /home/ec2-user/.cache/pip/wheels/92/a9/61/a37b5268b3b2ce5be8eb98bc6c39742d3cee57ec80515cdf92
Successfully built sagemaker
  Attempting uninstall: typing-extensions
    Found exi

In [13]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_chatglm_model")
local_model_path.mkdir(exist_ok=True)
model_name = "THUDM/chatglm-6b"
commit_hash = "f83182484538e663a03d3f73647f10f89878f438"

In [14]:
#执行这段，因为网络原因，可能会多次失败，需要反复执行
while True:
    try:
        snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)
        break
    except Exception:
        pass

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

### 2. 把模型拷贝到S3为后续部署做准备

In [9]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name


/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [14]:
s3_model_prefix = "LLM-RAG/workshop/LLM_chatglm_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_chatglm_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_chatglm_deploy_code
model_snapshot_path: LLM_chatglm_model/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438


In [17]:

#上传模型至S3
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_chatglm_model/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438/MODEL_LICENSE to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_model/MODEL_LICENSE
upload: LLM_chatglm_model/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438/.gitattributes to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_model/.gitattributes
upload: LLM_chatglm_model/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438/LICENSE to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_model/LICENSE
upload: LLM_chatglm_model/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438/configuration_chatglm.py to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_model/configuration_chatglm.py
upload: LLM_chatglm_model/models--THUDM--chatglm-6b/snapshots/f83182484538e663a03d3f73647f10f89878f438/config.json to s3://sagemaker-cn-north-1-507392672631

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [17]:
#适用于global
#inference_image_uri = (
#    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
#)

#中国区需要替换为下面的image_uri
inference_image_uri = (
     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117


In [20]:
!mkdir -p LLM_chatglm_deploy_code

In [102]:
%%writefile LLM_chatglm_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import pipeline, AutoModel, AutoTokenizer

def load_model(properties):
    logging.info(f"properties: {properties}")
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
   
    model = AutoModel.from_pretrained(model_location, trust_remote_code=True).half().cuda()
    
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None

def preprocess(text):
    text = text.replace("\n", "\\n").replace("\t", "\\t")
    return text

def postprocess(text):
    return text.replace("\\n", "\n").replace("\\t", "\t")

def answer(text, history=[], sample=True, top_p=0.45, temperature=0.01, model=None):
    text = preprocess(text)
    response, history = model.chat(tokenizer, text, history=history, temperature=temperature)
    
    return postprocess(response), history


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    input_data = inputs.get_as_json()
    logging.info(f"inputs: {input_data}")
    try:
        if 'history' not in input_data:
            history = []
        else:
            history = input_data['history']
        if 'temperature' not in input_data:
            temperature = 0.01
        else:
            temperature = input_data['temperature']
        response, history = answer(input_data['ask'], history=history, model=model)
        logging.info(f'====result {response}====')
        result = {"answer": response, "history" : history}
        return Output().add_as_json(result)
        
    except Exception as ex:
        traceback.print_exc(file=sys.stdout)
        logging.error(f"=================Exception================={ex}")
        
    result = {"answer": "No Answer", "history" : history}
    return Output().add_as_json(result)

Overwriting LLM_chatglm_deploy_code/model.py


In [89]:

print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

#使用打印出来的URL，替换下面一段代码中的S3Url

option.s3url ==> s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [90]:
%%writefile LLM_chatglm_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_model/

Overwriting LLM_chatglm_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [91]:
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1

Overwriting LLM_chatglm_deploy_code/requirements.txt


In [92]:
!rm model.tar.gz
!cd LLM_chatglm_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_chatglm_deploy_code

LLM_chatglm_deploy_code/
LLM_chatglm_deploy_code/serving.properties
LLM_chatglm_deploy_code/model.py
LLM_chatglm_deploy_code/requirements.txt


In [93]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/LLM_chatglm_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [94]:
from sagemaker.utils import name_from_base
import boto3

model_name = 'pytorch-inference-llm-v1'# name_from_base(f"chatglm") Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

pytorch-inference-llm-v1
Image going to be used is ---- > 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117
Created Model: arn:aws-cn:sagemaker:cn-north-1:507392672631:model/pytorch-inference-llm-v1


In [95]:
endpoint_config_name =model_name # f"{model_name}-config"
endpoint_name =model_name # f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
           # "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws-cn:sagemaker:cn-north-1:507392672631:endpoint-config/pytorch-inference-llm-v1',
 'ResponseMetadata': {'RequestId': '25c895d8-f7eb-4534-b33d-303b4f0bf7ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '25c895d8-f7eb-4534-b33d-303b4f0bf7ee',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Thu, 20 Jul 2023 07:39:07 GMT'},
  'RetryAttempts': 0}}

In [96]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws-cn:sagemaker:cn-north-1:507392672631:endpoint/pytorch-inference-llm-v1


#### 持续检测模型部署进度

In [97]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws-cn:sagemaker:cn-north-1:507392672631:endpoint/pytorch-inference-llm-v1
Status: InService


### 5. 模型测试

In [100]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
sagemaker_endpoint_name='pytorch-inference-llm-v1'
def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


In [101]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

KeyError: 'answer'

#### 清除模型Endpoint和config

In [ ]:
import boto3

def cleanup():
    # 创建一个sagemaker客户端
    sagemaker = boto3.client('sagemaker')

    # 删除模型
    sagemaker.delete_model(ModelName=sagemaker_endpoint_name)

    # 删除终端节点配置
    sagemaker.delete_endpoint_config(EndpointConfigName=sagemaker_endpoint_name)

    # 删除终端节点
    sagemaker.delete_endpoint(EndpointName=sagemaker_endpoint_name)

In [ ]:
#cleanup()
#!aws sagemaker delete-endpoint --endpoint-name pytorch-inference-llm-v1
#!aws sagemaker delete-endpoint-config --endpoint-config-name pytorch-inference-llm-v1
#!aws sagemaker delete-model --model-name pytorch-inference-llm-v1